In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
df_images_path_info = pd.read_csv('/kaggle/input/arts-info/3. images_info.csv')
df_images_path_info.fillna('', inplace=True)
df_images_path_info.head()

In [ ]:
df_images_info = pd.read_csv('/kaggle/input/arts-info/ObjectEng_.csv')
for col in df_images_info.columns:
    df_images_info[col].fillna('', inplace=True)
df_images_info.head()

In [ ]:
df_gis_info = pd.read_excel('/kaggle/input/arts-info/4. .xlsx')
for col in df_gis_info.columns:
    df_gis_info[col].fillna('', inplace=True)
df_gis_info.head()

In [ ]:
# Title-TranslatedObjectName
# Time-TranslatedPeriod TranslatedPeriodCentury
# Province-Province
# Theme-TranslatedSubcollection
# Material-TranslatedMaterial
# Descirption-TranslatedDescritpion TranslatedDescription

# 将文件4中的数据按照Province分组
province_2_inventory = df_gis_info.groupby('Province')['InventoryNum'].agg(set).to_dict()
print('省市-文物：', len(province_2_inventory))
print(province_2_inventory.keys())

In [ ]:
# 文物对应图片路径
# for image_path in df_images_path_info:
    # image_path['Relative Path'] = '../' + image_path['Relative Path'].split('/')[1]
    # df_images_path_info['Relative Path'] = df_images_path_info['Relative Path'].apply(lambda x: '../images/' + x.split('/')[1])
df_images_path_info['Relative Path'] = df_images_path_info['Relative Path'].apply(lambda x: '../images/' + x.split('/')[1])
# image_2_pict_path = df_images_path_info.groupby('Image Name')['Relative Path'].agg(str).to_dict()
image_2_pict_path = df_images_path_info.set_index('Image Name')['Relative Path'].to_dict()
print('文物=图片路径：', image_2_pict_path)

In [ ]:
# 声明InventoryInfo类，保存文物属性信息（名称、时间、省市、类别、材料、描述）
class InventoryInfo():
    def __init__(self, id='', title='', create_time='', province='', theme='', material='', description='', image_path='', image_circle=''):
        self.id = id
        self.title = title
        self.create_time = create_time
        self.province = province
        self.theme = theme
        self.material = material
        self.description = description
        self.image_path = image_path
        self.image_circle = image_circle
    
    def __str__(self):
        return f"InventoryInfo(id={self.id}, title={self.title}, create_time={self.create_time}, province={self.province}, theme={self.theme}, material={self.material}, description={self.description}, image_path={self.image_path}, image_circle={self.image_circle}\n)"

In [ ]:
id_2_inventory_info = {}

# 使用 iterrows() 遍历 DataFrame
for index, row in df_images_info.iterrows():
    if row['InventoryNumber'] in image_2_pict_path:
        image_path = image_2_pict_path[row['InventoryNumber']]
        temp = InventoryInfo(id=row['InventoryNumber'], title=row['TranslatedObjectName'], create_time=row['TranslatedPeriod'], province='', 
                             theme=row['TranslatedSubcollection'], material=row['TranslatedMaterial'], description=row['TranslatedDescription'], 
                             image_path=image_path, image_circle=image_path)
        id_2_inventory_info[row['InventoryNumber']] = temp

# print(id_2_inventory_info)

In [ ]:
# 省市-文物图片路径-文物信息
for province, inventory_list in province_2_inventory.items():
    for inventory in inventory_list:
        if inventory in id_2_inventory_info:
            id_2_inventory_info[inventory].province = province

# for title, info in id_2_inventory_info.items():
#     print(info)

In [ ]:
import json

# 遍历 id_2_inventory_info 字典，将 InventoryInfo 对象转换为 JSON 格式
inventory_info_list = []
for id, info in id_2_inventory_info.items():
    inventory_info_list.append(info.__dict__)

# 将 JSON 格式的数据写入文件
output_path = "/kaggle/working/inventory_info.json"
with open(output_path, "w") as json_file:
    print('开始写数据')
    json.dump(inventory_info_list, json_file, indent=4)
    print('结束写数据')